In [4]:
import selenium as selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
import subprocess
import json
import time as time

3.141.0


In [24]:
driver = webdriver.Chrome('../chromedriver_linux64/chromedriver')

# For selenium 4, refer to the link to launch the webdriver: https://pypi.org/project/webdriver-manager/

In [25]:
driver.get('https://www.epa.gov/outdoor-air-quality-data/download-daily-data')

In [9]:
f = open('polltant_names.json')
print(json.load(f))

['CO', 'Pb', 'NO2', 'Ozone', 'PM10', 'PM2.5', 'SO2']


In [10]:
f = open('state_names.json')
print(json.load(f))

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Puerto Rico']


In [31]:
selected_pollutant='SO2'
selected_state='Illinois'

In [33]:
# This cell will automatically download the CSV formatted data for the selected pollutant and state, for all available years

driver.refresh()

subprocess.run(['mkdir DataAQ/{}-{}'.format(selected_pollutant,selected_state)],shell=True)

select = Select(driver.find_element(By.NAME,'poll'))
pollutants=select.options

pollutant_names=[]
for p in pollutants:
    pollutant_names.append(p.text)

p_index=pollutant_names.index(selected_pollutant)

for p in range(p_index,p_index+1):

    select = Select(driver.find_element(By.NAME,'poll'))
    select.select_by_index(p)
    pollutant=select.first_selected_option.text
    time.sleep(2)

    select = Select(driver.find_element(By.NAME,'year'))
    years=select.options

    print(f'Data available from {years[2].text} to {years[-1].text}')

    for y in range(2,len(years)):

        select = Select(driver.find_element(By.NAME,'year'))
        select.select_by_index(y)
        year=select.first_selected_option.text
        time.sleep(2)

        select = Select(driver.find_element(By.NAME,'state'))
        states=select.options

        state_names=[]
        for s in states:
            state_names.append(s.text)

        s_index=state_names.index(selected_state)

        for s in range(s_index,s_index+1):

            select = Select(driver.find_element(By.NAME,'state'))
            select.select_by_index(s)
            state=select.first_selected_option.text
            time.sleep(2)

            driver.find_element(By.XPATH,'//*[@id="launch"]/input').click()  
            time.sleep(10)
            driver.find_element(By.PARTIAL_LINK_TEXT,'CSV').click()
            time.sleep(10)

            name=str(pollutant)+'-'+str(state)+'-'+str(year)
            subprocess.run(['mv ../ad_viz_plotval_data.csv DataAQ/{}-{}/{}'.format(selected_pollutant,selected_state,name)],shell=True) 
            print(f'{pollutant}-{year}-{state}')